## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-01-19-46-28 +0000,nypost,Here are the jobs that qualify for Trump’s ‘no...,https://nypost.com/2025/09/01/us-news/here-are...
1,2025-09-01-19-46-10 +0000,wapo,Nationwide Labor Day protests denounce billion...,https://www.washingtonpost.com/nation/2025/09/...
2,2025-09-01-19-37-12 +0000,bbc,Nestle fires boss after romantic relationship ...,https://www.bbc.com/news/articles/c1mpm9ee9p9o...
3,2025-09-01-19-30-13 +0000,nyt,"At Telluride, a Springsteen Biopic Means a Spr...",https://www.nytimes.com/2025/09/01/movies/tell...
4,2025-09-01-19-27-26 +0000,nypost,Mom of two dies slipping off 300-foot tower wh...,https://nypost.com/2025/09/01/world-news/mom-o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2444/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,32
170,china,15
162,afghanistan,9
66,gaza,8
163,earthquake,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
36,2025-09-01-17-25-00 +0000,wsj,The Trump administration has suspended visa ap...,https://www.wsj.com/politics/policy/trump-admi...,69
229,2025-08-31-19-58-46 +0000,nypost,Illinois Gov. Pritzker floats conspiracy theor...,https://nypost.com/2025/08/31/us-news/pritzker...,64
44,2025-09-01-17-09-57 +0000,nypost,Trump applauds ‘very popular’ DC Mayor Bowser ...,https://nypost.com/2025/09/01/us-news/presiden...,62
216,2025-08-31-21-45-30 +0000,nypost,Trump mulls ‘GREAT’ plan to pay Palestinians $...,https://nypost.com/2025/08/31/us-news/trump-mu...,62
119,2025-09-01-11-00-00 +0000,wsj,"The cash benefits of the new tax law, signed b...",https://www.wsj.com/politics/policy/companies-...,60


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
36,69,2025-09-01-17-25-00 +0000,wsj,The Trump administration has suspended visa ap...,https://www.wsj.com/politics/policy/trump-admi...
221,45,2025-08-31-21-23-44 +0000,nypost,Israel begins heavy airstrikes ahead of Gaza C...,https://nypost.com/2025/08/31/world-news/israe...
23,38,2025-09-01-18-23-50 +0000,nypost,Copper hits one-month peak on strong China fac...,https://nypost.com/2025/09/01/business/copper-...
25,35,2025-09-01-18-15-40 +0000,bbc,EU chief von der Leyen's plane hit by suspecte...,https://www.bbc.com/news/articles/c9d07z1439zo...
81,34,2025-09-01-14-47-18 +0000,nypost,Rudy Giuliani to remain in hospital after he w...,https://nypost.com/2025/09/01/us-news/rudy-giu...
99,32,2025-09-01-13-53-10 +0000,bbc,BBC reports from scene as Afghanistan earthqua...,https://www.bbc.com/news/videos/c3v3gg6zr3yo?a...
114,32,2025-09-01-11-51-28 +0000,nypost,Mom speaks out on hellish jail cell after bein...,https://nypost.com/2025/09/01/world-news/mom-w...
44,30,2025-09-01-17-09-57 +0000,nypost,Trump applauds ‘very popular’ DC Mayor Bowser ...,https://nypost.com/2025/09/01/us-news/presiden...
119,28,2025-09-01-11-00-00 +0000,wsj,"The cash benefits of the new tax law, signed b...",https://www.wsj.com/politics/policy/companies-...
32,25,2025-09-01-17-43-12 +0000,nypost,Rowdy ‘party pirates’ are taking over the wate...,https://nypost.com/2025/09/01/us-news/party-pi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
